# Import important libraries

In [38]:

import warnings
warnings.filterwarnings('ignore')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.compose import ColumnTransformer
from sklearn.utils import shuffle
sns.set_theme()
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import precision_recall_curve, roc_auc_score
from catboost import  CatBoostClassifier
from sklearn.metrics import fbeta_score
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
#visualization libraries
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.style as style
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.pyplot as plt, numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
from IPython.display import Image
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.offline as pyo
from plotly import tools
import seaborn as sns

import missingno as msno #to visualize missing data

from imblearn.over_sampling import SMOTE
import itertools


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix,precision_score,recall_score,roc_auc_score,f1_score,plot_confusion_matrix,plot_roc_curve,roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import LabelEncoder #label encoding for categorical columns


## Reading datasets & EDA



In [39]:
application_record = pd.read_csv("../input/credit-card-approval-prediction/application_record.csv", encoding = 'utf-8') 
credit_record = pd.read_csv("../input/credit-card-approval-prediction/credit_record.csv", encoding = 'utf-8') 

1. **application_record**: contains general customer data. Most likely, these are primary data and are stored in the storage based on received applications for the issuance of loan products.

2. **credit_record**: here you can find historical data on issued loan products, basic information, payment schedule for 

In [40]:
application_record.head(5)

In [41]:
application_record.info()

In [42]:
application_record.describe().T

In [43]:
for column in application_record.drop(columns=['ID']).columns:
    print(column)
    print(application_record[column].value_counts())
    

In [44]:
application_record.FLAG_MOBIL.value_counts() #every one has mobile 

In [45]:
print("Number of datapoints for application records: {}".format(len(application_record)))
print("Number of unique clients in dataset: {}".format(len(application_record.ID.unique())))


In [46]:
credit_record.head(5)

In [47]:
credit_record.info()

In [48]:
credit_record.describe().T

In [49]:
credit_record.STATUS.value_counts()

In [50]:
# data of active clients (open loan)

balance = credit_record[credit_record.ID.isin(credit_record[credit_record.MONTHS_BALANCE==0]['ID'])]
balance.groupby('STATUS')['ID'].count()


In [51]:
print("how many records match in two datasets: {}".format(len(set(application_record['ID']).intersection(set(credit_record['ID'])))) )

### Check for missing values

In [52]:
application_record.isna().sum()
# OCCUPATION_TYPE Has 134203 empty record

In [53]:
credit_record.isna().sum() #NO NULL Values


### Check for Duplicate records

In [54]:
application_record.duplicated().sum()


In [55]:
credit_record.duplicated().sum()


No duplicates in both datasets

# Data Visualization

In [56]:
#Now let's connect to the target attribute with general data from the appl dataset and the received credit history
df = application_record.merge(credit_record, on='ID', how='inner')


In [57]:
fig, ax = plt.subplots(figsize=(10,10))
cormat= df.corr()
mask = np.array(cormat)
mask[np.tril_indices_from(mask)] = False
sns.heatmap(cormat,mask=mask,annot = True)


In [58]:
def check_outliers(df):
    sns.set_style("whitegrid")
    for col in df.columns:
        if col != 'STATUS':
            fig = plt.figure(figsize=(8,7))
            sns.boxplot(x = 'STATUS', y = col , data = df)
            plt.show()
            
# Chart drawing function
def graffs(arr):
    fig, ax =plt.subplots(1,len(arr), figsize=(18,6))
    fig.autofmt_xdate()
    for index, category in enumerate(arr):

        #plt.subplot(1, index+1)   
        sns.countplot(data = df, x =category, hue='STATUS',ax=ax[index])
        ax[index].set_title(category)
        bars = ax[index].patches
        half = int(len(bars)/2)
        left_bars = bars[:half]
        right_bars = bars[half:]

    for left, right in zip(left_bars, right_bars):
        height_l = 0 if np.isnan(right.get_height()) else left.get_height() #Высота левого бара
        height_r = 0 if np.isnan(right.get_height()) else right.get_height() #Высота правого бара
        total = height_l + height_r #общая сумма строк
        if total > 0:

          #sign left 
          ax[index].text(left.get_x() + left.get_width()/2., height_l + 50, '{0:.0%}'.format(height_l/total), ha="center")
          #ax[index].text(left.get_x() + left.get_width()/2., height_l + 350, str(height_l), ha="center")

          #sign right
          #ax[index].text(right.get_x() + right.get_width()/2., height_r + 350, str(height_r), ha="center")
          ax[index].text(right.get_x() + right.get_width()/2., height_r + 50, '{0:.0%}'.format(height_r/total), ha="center")
    plt.tight_layout()        
    plt.show()

In [59]:
cols_to_plot = ["STATUS","CNT_CHILDREN","AMT_INCOME_TOTAL","DAYS_BIRTH","DAYS_EMPLOYED","MONTHS_BALANCE"]
check_outliers(df[cols_to_plot])

There are outliers in 2 columns.

* CNT_CHILDREN
* AMT_INCOME_TOTAL

In [60]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

r = 3
c = 3
fig, axs = plt.subplots(r, c, figsize=(20, 20))

i = 0
j = 0
for column in df.columns:
    if df[column].dtype == 'int64':
        sns.distplot(df[column], rug=True, bins=30, kde=True, hist=True, color='b', ax=axs[i, j])
        if(j == c-1):
            j = 0
            i += 1
        else:
            j += 1


In [61]:
fig, ax =plt.subplots(1,3, figsize=(19,5))
sns.boxenplot(data=df, x='AMT_INCOME_TOTAL', hue='STATUS',ax=ax[0])
sns.boxenplot(data=df, x='CNT_CHILDREN', hue='STATUS',ax=ax[1])
sns.boxenplot(data=df, x='CNT_FAM_MEMBERS', hue='STATUS',ax=ax[2])
plt.tight_layout()  
plt.show()


In [62]:
fig, axes = plt.subplots(1,3)

g1= application_record['CODE_GENDER'].value_counts().plot.pie(explode=[0.1,0.1],autopct='%1.1f%%',shadow=True, colors=["#3779C4","#B56F9B"],textprops = {'fontsize':12}, ax=axes[0])
g1.set_title("Customer Distribution by Gender")

g2= application_record['FLAG_OWN_CAR'].value_counts().plot.pie(explode=[0.1,0.1],autopct='%1.1f%%',shadow=True,colors=["#3779C4","#B56F9B"],textprops = {'fontsize':12}, ax=axes[1])
g2.set_title("Car Ownership")

g3= application_record['FLAG_OWN_REALTY'].value_counts().plot.pie(explode=[0.1,0.1],autopct='%1.1f%%',shadow=True,colors=["#3779C4","#B56F9B"],textprops = {'fontsize':12}, ax=axes[2])
g3.set_title("Realty Ownership")

fig.set_size_inches(14,5)

plt.tight_layout()

plt.show()

In [63]:
sns.set_theme()
graffs(['NAME_FAMILY_STATUS','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE'])


In [64]:
credit_record.groupby('STATUS').count().plot(kind='bar')

# Data Preprocessing & Feature Engineering

In [65]:
# determination of the month of opening a loan (the expected date of issuing a loan) and the current month
credits_history = balance.groupby(['ID']).agg(
   start_month=  ('MONTHS_BALANCE', min), end_month=('MONTHS_BALANCE', max), 
   current_status =('STATUS', 'first'),
   last_status = ('STATUS', 'last')
    ).reset_index()


In [66]:
credits_history.head()

In [67]:

# determination of the month of opening a loan (the expected date of issuing a loan) and the current month
credits_history = balance.groupby(['ID']).agg(
   start_month=  ('MONTHS_BALANCE', min), end_month=('MONTHS_BALANCE', max), 
   current_status =('STATUS', 'first'),
   last_status = ('STATUS', 'last')
    ).reset_index()
credits_history['duration'] = round(abs(credits_history.start_month) / 12)
credits_history = pd.merge(credits_history, balance, how='left', on='ID')
credits_history = credits_history[~credits_history.current_status.isin(["X"])]
credits_history


In [68]:
# grouping and counting statuses by month
bad_month = credits_history.groupby(['MONTHS_BALANCE', 'STATUS']).size().reset_index(name='counts')
bad_month.drop(bad_month[bad_month.STATUS.isin(["X","C"])].index,inplace=True)
bad_month['MONTHS_BALANCE'] =  abs(bad_month['MONTHS_BALANCE'])
bad_month

Thus, a level 0 delay occurs in most clients, during the entire loan period, the peak falls on the first 5 months. Delay of the first level is also not uncommon to have a similar distribution pattern with zero, but with a lower frequency.

Overdue payments of levels 2, 3, 4, 5 Significantly fewer, but they are of a more prolonged nature. In long-term loans, users with a history of more than 5 years after about 50 months have no such delays at all.

Based on this data, we will mark up the dataset with general data as follows.

A debt of 1-5 level formed within 18 months is a "bad client"

In [69]:
def chek(arr):
    s = 1
    for i,j in enumerate(arr):
        if i <= 17 and j in bad:
            s = 0
            break   
    return s  

In [70]:
bad = ["1", "2", "3", "4", "5"]
target = credits_history.groupby('ID')['STATUS'].apply(chek).reset_index()
#Now let's connect to the target attribute with general data from the appl dataset and the received credit history
df = application_record.merge(target, on='ID', how='inner')





In [71]:
df = df.merge(credits_history.groupby('ID')[['duration','start_month']].first(), on='ID', how='left')
df = df.drop(df[df.STATUS.isna()].index)

In [72]:
df.groupby(['NAME_INCOME_TYPE','OCCUPATION_TYPE']).count().sort_values('ID').groupby(level=0).tail(1)['ID']
df.loc[df['NAME_INCOME_TYPE'] == 'Working','OCCUPATION_TYPE']
df.loc[df['NAME_INCOME_TYPE'] == 'Working','OCCUPATION_TYPE'] = \
df.loc[df['NAME_INCOME_TYPE'] == 'Working','OCCUPATION_TYPE'].fillna(value='Laborers')
df.loc[df['NAME_INCOME_TYPE'] == 'Commercial associate','OCCUPATION_TYPE'] = \
df.loc[df['NAME_INCOME_TYPE'] == 'Commercial associate','OCCUPATION_TYPE'].fillna(value='Laborers')
df.loc[df['NAME_INCOME_TYPE'] == 'State servant','OCCUPATION_TYPE'] = \
df.loc[df['NAME_INCOME_TYPE'] == 'State servant','OCCUPATION_TYPE'].fillna(value='Core staff')
df.loc[df['NAME_INCOME_TYPE'] == 'Student','OCCUPATION_TYPE'] = \
df.loc[df['NAME_INCOME_TYPE'] == 'Student','OCCUPATION_TYPE'].fillna(value='Laborers')
df.loc[df['NAME_INCOME_TYPE'] == 'Pensioner','OCCUPATION_TYPE'] = \
df.loc[df['NAME_INCOME_TYPE'] == 'Pensioner','OCCUPATION_TYPE'].fillna(value='Core staff')











In [73]:
df[df.drop(['ID'],axis=1).duplicated(keep=False)==True].groupby('STATUS')['ID'].count()


In [74]:
df.drop(df[df.drop(['ID'],axis=1).duplicated()==True].index, inplace=True)


In [75]:
df[df.drop(['ID','CODE_GENDER'],axis=1).duplicated(keep=False)==True].sort_values(by='DAYS_BIRTH')
df.drop(df[df.drop(['ID','CODE_GENDER'],axis=1).duplicated(keep=False)==True].index,inplace=True)
df['credit_counts'] = 1
columns = df.drop(['ID','credit_counts','STATUS','duration','start_month'],axis=1).columns.tolist()
df['credit_counts'] = df.groupby(columns)['credit_counts'].apply(np.cumsum)
df.credit_counts.describe()



In [79]:
import datetime
def my_years_my_wealth(days):
    cur_date = datetime.datetime.now()
  #print(pd.to_datetime(days, origin=cur_date, unit='D').year)
    if days > -365:
        return 0
    return cur_date.year - pd.to_datetime(days, origin=cur_date, unit='D').year

df['income_category'] = pd.qcut(df.AMT_INCOME_TOTAL,5,labels=[1, 2, 3, 4, 5])
df['income_per_person'] = df.AMT_INCOME_TOTAL / df.CNT_FAM_MEMBERS
df['DAYS_EMPLOYED'] = df.DAYS_EMPLOYED.apply(my_years_my_wealth)
df['DAYS_BIRTH'] = df.DAYS_BIRTH.apply(my_years_my_wealth)
df['age_when_start'] = df.DAYS_BIRTH - df.duration
df['emploed_when_start'] = df.DAYS_EMPLOYED - df.duration
df.drop(columns=['DAYS_BIRTH','DAYS_EMPLOYED'], inplace=True)




In [80]:
def Y_or_N(x):
    return [1,0][x.upper() == 'Y']

df.FLAG_OWN_CAR = df.FLAG_OWN_CAR.apply(Y_or_N)
df.loc[df['emploed_when_start'] < 0, 'emploed_when_start'] = -1
df.FLAG_OWN_REALTY = df.FLAG_OWN_REALTY.apply(Y_or_N)



In [81]:
#Remove unnecessary data from the dataset
df_features = df.drop(['ID','STATUS','start_month','duration',
                       'AMT_INCOME_TOTAL'], axis=1)
df_target = df['STATUS']
num_features = ['income_category','age_when_start','emploed_when_start','CNT_FAM_MEMBERS','credit_counts','income_per_person']


In [82]:
cat_features=['CODE_GENDER','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS',
              'NAME_HOUSING_TYPE','OCCUPATION_TYPE']
df_features[num_features] = df_features[num_features].astype('int64')



## MODEL

In [84]:
X_train, X_test, y_train, y_test = train_test_split(df_features, df_target, test_size=0.2, random_state=42)


In [86]:
from sklearn.compose import ColumnTransformer
from sklearn.utils import shuffle
sns.set_theme()
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import precision_recall_curve, roc_auc_score
from catboost import  CatBoostClassifier
from sklearn.metrics import fbeta_score
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score


# upsample
def upsample(features, target, repeat):
    
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
 
    features_upsampled = pd.concat([features_ones] + [features_zeros] * repeat)
    target_upsampled = pd.concat([target_ones] + [target_zeros] * repeat)
    
    features_upsampled = shuffle(features_upsampled, random_state=42)
    target_upsampled = shuffle(target_upsampled, random_state=42)
    
    return features_upsampled, target_upsampled
features_upsampled, target_upsampled = upsample(X_train, y_train, 8)
print(features_upsampled.shape)
print(target_upsampled.mean())

In [87]:
# downsample
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_sample = features_ones.sample(frac=fraction, random_state=42)
    target_sample = target_ones.sample(frac=fraction, random_state=42)
    
    features_downsampled = pd.concat([features_sample] + [features_zeros])
    target_downsampled = pd.concat([target_sample] + [target_zeros])
    
    features_downsampled = shuffle(features_downsampled, random_state=42)
    target_downsampled = shuffle(target_downsampled, random_state=42)
    
    return features_downsampled, target_downsampled
features_downsample, target_downsample = downsample(X_train, y_train, 0.17)
print(features_downsample.shape)
print(target_downsample.mean())


In [90]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor

def results(model, scor_type, X, y, sample):
    predict = model.predict(X)
    tn, fp, fn, tp = confusion_matrix(y, predict).ravel()
    f1 = f1_score(y, predict)
    roc = roc_auc_score(y, model.predict_proba(X)[:,1])
    f_beta = fbeta_score(y, predict,beta=0.5)
    return ({'dataset': sample,
                      'tune_metric': scor_type,
                      'roc_auc_score': roc,
                      'f1_beta_score': f1,
                      'f1_score': f1,
                      'tn': tn,
                      'fp': fp,
                      'fn': fn,
                      'tp': tp})
columns = ['dataset','tune_metric','roc_auc_score','f1_beta_score','f1_score', 'tn','fp','fn','tp']
res_metrics = pd.DataFrame(columns=columns)
categories = []
for cat in cat_features:
    categories.append(df[cat].unique().tolist())


In [92]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import  GridSearchCV
from sklearn.linear_model import LogisticRegression

preprocessor = ColumnTransformer(
    [
        ("onehot_categorical", OneHotEncoder(categories=categories, drop='first'), cat_features),
        ('scaler', StandardScaler(), num_features)
    ], remainder='passthrough'
)
pipeline = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('model', LogisticRegression())
    ]
)
preprocessor.fit_transform(features_downsample)[:1]


In [93]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier



params = [
    {
        'model': [LogisticRegression()]
      
        
  },
  {
        'model': [XGBClassifier()],
        'model__n_estimators': [i for i in range(1, 70,5)],
        'model__max_depth': [i for i in range(1, 20,2)],
        'model__random_state': [42],
        'model__criterion':['gini', 'entropy'],
        'preprocessor__scaler': ['passthrough']
      
        
  },
  {   
        'model': [CatBoostClassifier()],   
        'model__solver': ['liblinear'],
        #'model__penalty': ['l1', 'l2'],
        #'model__C': [0.01, 0.1, 1, 10],
        'model__random_state': [42],
        'preprocessor__scaler': [StandardScaler()]
   
  }

]
grid = GridSearchCV(pipeline, param_grid=params, verbose=False,  
                                                 cv=3, 
                                                 scoring='roc_auc')

In [94]:
grid = GridSearchCV(pipeline, param_grid=params, verbose=False,  
                                                 cv=3, 
                                                 scoring='roc_auc')


In [95]:
grid.fit(features_upsampled, target_upsampled)


In [ ]:
pd.DataFrame(grid.cv_results_).sort_values(by="mean_test_score", ascending=False)[[
                                                                                  'param_model__max_depth',
                                                                                  'param_model__n_estimators',
                                                                                  'mean_test_score',
                                                                                  'param_model']].head(3)


In [ ]:
grid.scorer_


In [ ]:
res_metrics = res_metrics.append(results(grid.best_estimator_, grid.scorer_,
                                         X_test,y_test,
                                         'test/upsampled'),ignore_index=True)


In [ ]:
res_metrics


In [ ]:
probabilities_valid = grid.best_estimator_.predict_proba(X_test)


In [ ]:
probabilities_one_test = probabilities_valid[:, 1]


In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, probabilities_one_test) 

plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, linestyle='-')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-Curve')


In [ ]:
for threshold in np.arange(0, 0.95, 0.05):
    predicted_valid =  (probabilities_one_test > threshold)*1
    precision = precision_score(y_test,predicted_valid)
    recall = recall_score(y_test,predicted_valid)
    tn, fp, fn, tp = confusion_matrix(y_test, predicted_valid).ravel()
    print("Recall = {:.2f} | Percision = {:.3f}, completeness = {:.3f}".format(
        threshold, precision, recall))
    print('TN = {}| FP {}| FN {} | TP {} |'.format(tn, fp, fn, tp))

In [ ]:
cbr = CatBoostClassifier(eval_metric='AUC', iterations=1000, random_state=42,
                         learning_rate=0.05, depth=6, verbose=1000)
cbr.fit(features_upsampled ,target_upsampled, cat_features=cat_features , eval_set=(X_test, y_test), plot=True)
catboost_feachures_important=pd.DataFrame({'feature_importance': cbr.feature_importances_, 
              'feature_names': X_train.columns}).sort_values(by=['feature_importance'], 
                                                           ascending=False)
res_metrics = res_metrics.append(results(cbr, 'roc_auc',
                                         X_test,y_test,
                                         'test/upsampled'),ignore_index=True)
probabilities_valid = cbr.predict_proba(X_test)
probabilities_one_test = probabilities_valid[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, probabilities_one_test) 

plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, linestyle='-')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-Curve')

